<a href="https://colab.research.google.com/github/Pushkarp26/Text-Summarization-with-Punctuations/blob/main/Text%20summarization%20code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing Libraries

!pip install pyLDAvis
# !apt-get install ocrmypdf -q
!pip install pdfplumber -q
!pip install transformers

# Importing modules
import pdfplumber
import re
import random
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet') 
import pandas as pd
import numpy as np
import string,pprint
import spacy
!pip install gensim==3.8.0
import gensim
from gensim import corpora
from transformers import GPT2Tokenizer, GPT2LMHeadModel
# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
## reading in the PDF file
with pdfplumber.open(r'/content/drive/MyDrive/Non Disclosure agreemnet.pdf') as pdf:
  no_of_pages = len(pdf.pages)
  print("The total number of pages of the document are: "+ str(no_of_pages))
 
 
for i in range(no_of_pages):
  with pdfplumber.open(r'/content/drive/MyDrive/Non Disclosure agreemnet.pdf') as pdf:
    page_content = pdf.pages[i]
    globals()['page%s' %i] = page_content.extract_text()

page0 = page0.replace("\nCreate your own automated PDFs with JotForm PDF Editor","")
page1 = page1.replace("\nCreate your own automated PDFs with JotForm PDF Editor","")
page2 = page2.replace("\nCreate your own automated PDFs with JotForm PDF Editor","")
page3 = page3.replace("\nIN WITNESS WHEREOF, the Parties have set their hands on the date specified.\nReceiving Party Disclosing Party\nPraesent id ABC Company\n20 International P, Worce, Ma, 01654 291 Arbutus Drive, Miami, FL, 12345 \nUnited States United States\nRepresentative:  Representative: \nMeggy Ciepluch John Smith\nSignature:\nSignature: \nDate Signed: 11/11/1996 Date Signed: 11/11/1996\nCreate your own automated PDFs with JotForm PDF Editor 4","")
 
for i in range (no_of_pages):
  text = globals()['page%s' %i]
  text = text.replace('\n',' ')
  text = re.sub(r"(XI+V*.)*(VI*.)*(I?X.)*(II+.)*","",text)
  text = text.lower()
  text = re.sub(r"[\@#$%(){}/`~:<>+=-]","",text)
  text = re.sub(r"[\"]","",text)
  text = re.sub(r"\d","",text)
  text = re.sub(r"(XI+V*.)*(VI*.)*(I?X.)*(II+.)*","",text)
  stop_words = stopwords.words('english')
  word_tokens = word_tokenize(text)
  tokens_without_sw = [word for word in word_tokens if not word in stopwords.words()]
  filtered_text = (" ").join(tokens_without_sw)
  lemmatizer = WordNetLemmatizer() 
  word_tokens = word_tokenize(filtered_text)
  lemmatized_string = ' '.join([lemmatizer.lemmatize(words) for words in word_tokens])
  final_list = word_tokenize(lemmatized_string)
  globals()['page%s' %i] = final_list
 
doc_list = [page0,page1,page2,page3]
 
print(doc_list)

The total number of pages of the document are: 4
[['disclosure', 'agreement', 'disclosure', 'confidentiality', 'agreement', 'agreement', 'entered', 'abc', 'company', ',', 'hereinafter', 'known', 'disclosing', 'party', ',', 'undersigned', 'receiving', 'party', 'receiving', 'party', ',', 'collectively', 'party', 'known', 'party', '.', 'whereas', ',', 'disclosing', 'party', 'shall', 'disclose', 'deliver', 'tangible', 'intangible', 'information', 'receiving', 'party', 'may', 'considered', 'confidential', 'proprietary', 'information', ';', 'whereas', ',', 'disclosing', 'party', 'desire', 'assured', 'proprietary', 'information', 'owned', 'shall', 'disclosed', 'third', 'party', 'without', 'consent', 'shall', 'maintained', 'protected', 'receiving', 'party', ';', ',', 'therefore', ',', 'consideration', 'mutual', 'promise', 'covenant', 'included', 'agreement', ',', 'party', 'agree', 'follows', 'i.', 'definition', '.', 'confidential', 'information', 'shall', 'refer', 'kind', 'information', 'wheth

In [3]:
doc = [str(" ".join(i)) for i in doc_list]
doc

['disclosure agreement disclosure confidentiality agreement agreement entered abc company , hereinafter known disclosing party , undersigned receiving party receiving party , collectively party known party . whereas , disclosing party shall disclose deliver tangible intangible information receiving party may considered confidential proprietary information ; whereas , disclosing party desire assured proprietary information owned shall disclosed third party without consent shall maintained protected receiving party ; , therefore , consideration mutual promise covenant included agreement , party agree follows i. definition . confidential information shall refer kind information whether oral written made provided disclosing party receiving party vice versa . could written oral communication may refer patent , copyright , trademark , trade secret . information agreement declared constituted confidential disclosing party , regardless whether information provided date agreement shall , limite

###BART Model

In [4]:
from transformers import pipeline

In [ ]:
# Initialize the HuggingFace summarization pipeline
summarizer = pipeline("summarization", model='facebook/bart-large-cnn', tokenizer='facebook/bart-large-cnn')
summarized = summarizer(doc,min_length = round(0.1 * len(text.split(' '))), max_length = round(0.2 * len(text.split(' '))))

# Print summarized text
print(summarized)

In [12]:
summarized

[{'summary_text': 'Confidential information shall refer kind information whether oral written made provided disclosing party receiving party vice versa. could written oral communication may refer patent , copyright , trademark , trade secret. Information agreement declared constituted confidential disclosing party'},
 {'summary_text': 'Technology refers technical scientific information material produced formulation made disclosing party information publicly used. Receiving party shall hold trust confidence information possessed said party shall disclose party use benefit without consent disclosing party.'},
 {'summary_text': ' confidential information disclosed agreement shall returned to disclosing party. return confidential information. upon termination agreement andor upon request disclosing party, confidential information may be destroyed. Neither party may assign delegate right , delegate obligation hereunder'},
 {'summary_text': ' agreement understood complete agreement party her

###T5 Model


In [ ]:
!pip install sentencepiece

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')
Preprocessed_text = "summarize: " + str(doc)
tokens_input = tokenizer.encode(Preprocessed_text,return_tensors="pt", max_length=512, truncation=True)
summary_ids = model.generate(tokens_input,
                             min_length=60,
                             max_length=180,
                             length_penalty=4.0) 
summary = tokenizer.decode(summary_ids[0])

In [13]:
summary

'<pad> disclosing party shall disclose deliver tangible intangible information. whereas, disclosing party desire assured proprietary information owned shall disclosed third party without consent shall maintained protected receiving party. whereas, disclosing party may disclose deliver tangible intangible information. whereas, disclosing party may disclose deliver tangible intangible information.</s>'

###BERT- extractive summarizer


In [11]:
!pip install bert-extractive-summarizer

In [9]:
from summarizer import TransformerSummarizer

In [10]:
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
summerize = ''.join(GPT2_model(str(doc), min_length=100, max_length=260))
summerize

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2-medium and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'h.12.attn.masked_bias', 'h.13.attn.masked_bias', 'h.14.attn.masked_bias', 'h.15.attn.masked_bias', 'h.16.attn.masked_bias', 'h.17.attn.masked_bias', 'h.18.attn.masked_bias', 'h.19.attn.masked_bias', 'h.20.attn.masked_bias', 'h.21.attn.masked_bias', 'h.22.attn.masked_bias', 'h.23.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


"['disclosure agreement disclosure confidentiality agreement agreement entered abc company , hereinafter known disclosing party , undersigned receiving party receiving party , collectively party known party . business operation information refers information business operation party disclosing information , including number personnel , financial activity within , information vendor , provider , partner openly declared party public . . product information refers specification product produced disclosing party well product resulting related work project performed performed disclosing party client , information product research development ; . confidential information may likewise include information disclosed party disclosing party protected nondisclosure agreement accidentally , incidentally , acquired receiving party whether directly indirectly . receiving party , upon acquiring confidential information shall hold trust confidence information possessed said party shall disclose party u